#//*[@id="main"]/div[2]/div[9]

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
url = "https://pvpoketw.com/rankings/all/1500/leads/"
driver.get(url)
driver.minimize_window()
# 等待網頁加載完成
time.sleep(5)
table = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[9]')
text = table.text
text = text.replace('\n†\n,', ',')
text = text.replace('\n†\n', '\n')
text = text.split('\n')


In [12]:
from flask import Flask, render_template
app = Flask(__name__)

table_rows = []
@app.route('/')
def display_table():

    
    # 每三項為一組，以並排顯示
    for i in range(0, len(text), 3):
        name = text[i]
        moves = text[i + 1]
        
        score = text[i + 2]
        
        
        table_rows.append({
            'name': name,
            'moves': moves,
            'score': score
        })

    return render_template('index.html', rows=table_rows)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Jun/2023 21:48:53] "GET / HTTP/1.1" 200 -


In [13]:
import pyodbc

# 建立與 SQL Server 的連線
conn = pyodbc.connect("Driver={SQL Server};SERVER=MSI;DATABASE=poke;UID=sa;PWD=123")

# 確認連線成功
if conn:
    print("OK")

# 建立 cursor 來執行 SQL 語法
cursor = conn.cursor()




OK


In [ ]:
cursor.execute('''
		CREATE TABLE cp1500 (
            name nvarchar(50),
            moves nvarchar(150),
            score nvarchar(50)
			)
               ''')

conn.commit()

In [ ]:
#更換moves欄位的逗號為底線
for row in table_rows:
    moves = row['moves'].replace(',', '_')
    row['moves'] = moves

In [ ]:
#插入資料
insert_sql = "INSERT INTO cp1500 (name, moves, score) VALUES (?, ?, ?)"

for row in table_rows:
    values = (row['name'], row['moves'], row['score'])
    cursor.execute(insert_sql, values)

# 提交資料庫交易
conn.commit()


In [ ]:
cursor.execute('SELECT * FROM cp1500')

# 獲取查詢結果
result = cursor.fetchall()

# 輸出結果
for row in result:
    print(row)

# 關閉連線
cursor.close()
conn.close()
